<a href="https://colab.research.google.com/github/AnanyaGodse/Intelligent-Systems/blob/main/IS_Exp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name: Ananya Godse SAP ID: 60009220161 Batch: D1-1**

###**LAB 3 - Heuristic Search**

In [ ]:
import copy
import heapq

In [ ]:
start_state = [['A', 'B', 'C', 'D', 'T'], ['E', 'F', 'T']] # T represents the table

In [ ]:
goal_state = [['A', 'E', 'B', 'C', 'D', 'T'], ['F', 'T']]

**Function to check if the current state is the goal state:**

In [ ]:
def goal_test(state, goal_state):
  if state == goal_state:
    return True
  else:
    return False

**Function to generate next possible states:**:

In [ ]:
def move_gen(state):
    next_states = []
    moves = []

    # Check if each pile has at least one block (ignoring 'T' at the bottom)
    block1 = state[0][0] if len(state[0]) > 1 else None
    block2 = state[1][0] if len(state[1]) > 1 else None

    # If block1 exists, we can move it
    if block1:
        moves.append("block1-on-table")
        moves.append("block1-on-block2")

    # If block2 exists, we can move it
    if block2:
        moves.append("block2-on-table")
        moves.append("block2-on-block1")

    for move in moves:
        new_state = copy.deepcopy(state)

        if move == "block1-on-table":
            new_state[0].remove(block1)
            new_state.append([block1, 'T'])  # Move block1 onto the table
            next_states.append(new_state)
        elif move == "block1-on-block2" and block2:
            new_state[0].remove(block1)
            new_state[1].insert(0, block1)  # Move block1 onto block2
            next_states.append(new_state)
        elif move == "block2-on-table":
            new_state[1].remove(block2)
            new_state.append([block2, 'T'])  # Move block2 onto the table
            next_states.append(new_state)
        elif move == "block2-on-block1" and block1:
            new_state[1].remove(block2)
            new_state[0].insert(0, block2)  # Move block2 onto block1
            next_states.append(new_state)

    return next_states



In [ ]:
next_states = move_gen(start_state)
next_states


[[['B', 'C', 'D', 'T'], ['E', 'F', 'T'], ['A', 'T']],
 [['B', 'C', 'D', 'T'], ['A', 'E', 'F', 'T']],
 [['A', 'B', 'C', 'D', 'T'], ['F', 'T'], ['E', 'T']],
 [['E', 'A', 'B', 'C', 'D', 'T'], ['F', 'T']]]

In [ ]:
test_state = [['A', 'C', 'D', 'T'], ['F', 'E', 'T']]
next_states = move_gen(test_state)
next_states

[[['C', 'D', 'T'], ['F', 'E', 'T'], ['A', 'T']],
 [['C', 'D', 'T'], ['A', 'F', 'E', 'T']],
 [['A', 'C', 'D', 'T'], ['E', 'T'], ['F', 'T']],
 [['F', 'A', 'C', 'D', 'T'], ['E', 'T']]]

**Heuristic 1 - +1 for every block on top of a correct block and -1 for every block on top of a wrong block.**

In [ ]:
def heuristic_1(state, goal_state):
    score = 0

    for i in range(len(state)):  # iterate over each pile in the state
        for j in range(len(state[i]) - 1):  # iterate over blocks in each pile, ignoring 'T'
            if state[i][j] == 'T':
                continue
            # Check if the current block is on the correct block as per the goal state
            current_block = state[i][j]
            below_current = state[i][j+1]

            # Find where this block and below it should be in the goal state
            for gi in range(len(goal_state)):
                if current_block in goal_state[gi]:
                    correct_pile = goal_state[gi]
                    if (j + 1 < len(correct_pile)) and (below_current == correct_pile[correct_pile.index(current_block) + 1]):
                        score += 1  # Block is correctly placed on top of the correct block
                    else:
                        score -= 1  # Block is not on the correct block
    return score




In [ ]:
h1_score = heuristic_1(test_state, goal_state)
print("Heuristic 1 Score:", h1_score)

Heuristic 1 Score: -1


**Heuristic 2 - + n for every block on top of the correct pile (n = no of blocks in the pile) and -n for every block not on the correct pile.**

In [ ]:
def heuristic_2(state, goal_state):
    score = 0

    for i in range(len(state)):  # iterate over each pile in the state
        for j in range(len(state[i]) - 1):  # iterate over blocks in each pile, ignoring 'T'
            if state[i][j] == 'T':
                continue
            # Get current block
            current_block = state[i][j]

            # Find where this block should be in the goal state
            for gi in range(len(goal_state)):
                if current_block in goal_state[gi]:
                    correct_pile = goal_state[gi]
                    if i == gi:  # If block is on the correct pile
                        score += j  # Add +n, where n = number of blocks below it
                    else:
                        score -= j  # If block is on the wrong pile, subtract n (number of blocks below it)
    return score


In [ ]:
h2_score = heuristic_2(test_state, goal_state)
print("Heuristic 2 Score:", h2_score)

Heuristic 2 Score: 2


**Best-First Search (using priority queue based on heuristic 2):**

In [ ]:
def best_first_search(start_state, goal_state):
    # Priority queue initialized with the start state
    frontier = []
    heapq.heappush(frontier, (heuristic_2(start_state, goal_state), start_state))

    explored = set()

    while frontier:
        # Get the state with the lowest heuristic score
        current_cost, current_state = heapq.heappop(frontier)

        # Check if it's the goal state
        if goal_test(current_state, goal_state):
            return current_state, current_cost

        explored.add(tuple(map(tuple, current_state)))  # Add the current state to explored

        # Generate possible next states
        next_states = move_gen(current_state)

        for next_state in next_states:
            if tuple(map(tuple, next_state)) not in explored:
                heapq.heappush(frontier, (heuristic_2(next_state, goal_state), next_state))

    return None, float('inf')  # Return failure if no solution is found


**Hill Climbing (using heuristic 2):**

In [ ]:
def hill_climbing(start_state, goal_state):
    current_state = start_state
    current_cost = heuristic_2(current_state, goal_state)

    while True:
        next_states = move_gen(current_state)
        next_states_with_costs = [(heuristic_2(state, goal_state), state) for state in next_states]

        # Find the best next state (state with highest heuristic score)
        best_cost, best_state = max(next_states_with_costs, key=lambda x: x[0])

        # If the current state is better than or equal to the best next state, stop (local maxima)
        if best_cost <= current_cost:
            return current_state, current_cost

        # Move to the next best state
        current_state = best_state
        current_cost = best_cost

        # If we reach the goal state, return
        if goal_test(current_state, goal_state):
            return current_state, current_cost

In [ ]:
# Run Best-First Search
bfs_result, bfs_cost = best_first_search(start_state, goal_state)
print("Best-First Search Result:", bfs_result)
print("Best-First Search Cost:", bfs_cost)

Best-First Search Result: None
Best-First Search Cost: inf


Best First Search wasn't able to find an answer. It might have reached a dead end. The effectiveness of best first search depends on the choice of heuristic.

In [ ]:
# Run Hill Climbing
hc_result, hc_cost = hill_climbing(start_state, goal_state)
print("Hill Climbing Result:", hc_result)
print("Hill Climbing Cost:", hc_cost)

Hill Climbing Result: [['F', 'E', 'A', 'B', 'C', 'D', 'T'], ['T']]
Hill Climbing Cost: 15


Hill Climbing did give us an answer but it is not the goal state. Hill Climbing is prone to getting stuck in local maxima due to ineffective heuristic functions.